---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind


# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [13]:
def get_list_of_university_towns():
    
   # towns = pd.read_fwf('university_towns.txt', sep=" ", header = None)
    df = pd.read_csv('university_towns.txt', sep='\n', header=None, names=["0"])
    pd.options.display.max_rows=1000
    
    df['RegionName'] = df['0'].apply(lambda x: x.split(
    '(')[0].strip() if x.count('[edit') == 0 else np.NaN)
    
    df['State'] = df['0'].apply(lambda x: x.split('[')[0].strip(
    ) if x.count('[edit') > 0 else np.NaN).fillna(method="ffill")

    df = df.dropna().drop('0', axis=1).reindex(
    columns=[ 'State','RegionName']).reset_index(drop=True)
    

    
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    return df
get_list_of_university_towns()




,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [4]:
def get_dfq():
    
    dfq = pd.read_excel('gdplev.xls',  skiprows=7)
    cols = ['Unnamed: 4','Unnamed: 6']
    dfq=dfq[cols]
    dfq.rename(columns={'Unnamed: 4':'quarter', 'Unnamed: 6':'GDP'}, inplace=True)
    
    dfq= dfq.drop(dfq.index[0:212])
    dfq.reset_index(drop=True, inplace=True)
    
    dfq['GDP_next'] = dfq['GDP'].shift(-1) 
    dfq['GDP_prev'] = dfq['GDP'].shift(+1)
    
    dfq['GDP_prev2'] = dfq['GDP_prev'].shift(+1)
    
    dfq['pot_reces_start'] =( dfq['GDP_prev']>dfq['GDP']) &( dfq['GDP']>dfq['GDP_next'])
    dfq['pot_reces_end'] = (dfq['GDP_prev2']< dfq['GDP_prev'])& (dfq['GDP_prev']<dfq['GDP']) 

    return dfq
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    dfq = get_dfq()
    recession = False
    for x in range (len(dfq)):
        if dfq.iloc[x]['pot_reces_start'] ==True and recession ==False:
            recession = True
          
            return dfq.iloc[x]['quarter']
    
get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    
    dfq=get_dfq()
    recession = False
    for x in range (len(dfq)):
        if dfq.iloc[x]['pot_reces_start'] ==True and recession ==False:
            recession = True
        if dfq.iloc[x]['pot_reces_end']==True and recession ==True:
            return dfq.iloc[x]['quarter']
get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    
    dfq = get_dfq()
    start = get_recession_start()
    start_index = dfq[dfq['quarter'] == start].index.tolist()[0]
    
    end=get_recession_end()
    end_index = dfq[dfq['quarter'] == end].index.tolist()[0]
   
    gdp= dfq.iloc[start_index:end_index+1]
    bottom = gdp['GDP'].min()
    bottom_index= gdp[gdp["GDP"]== bottom].index.tolist()[0]-start_index
    
    return gdp.iloc[bottom_index]['quarter']

    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
get_recession_bottom()

'2009q2'

In [7]:
def convert_housing_data_to_quarters():
    
    dfh = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    dfh['State'] = dfh['State'].replace({'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'})
    dfh= dfh.set_index(["State","RegionName"])
    dfh = dfh.iloc[:,49:250]
    
    def quarters(col):
        if col.endswith(("01", "02", "03")):
            s = col[:4] + "q1"
        elif col.endswith(("04", "05", "06")):
            s = col[:4] + "q2"
        elif col.endswith(("07", "08", "09")):
            s = col[:4] + "q3"
        else:
            s = col[:4] + "q4"
        return s  
    housing = dfh.groupby(quarters, axis = 1).mean()
    housing = housing.sort_index()
    return housing
    
    #Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
    
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''

convert_housing_data_to_quarters()

2000q1        2000q2  \
State         RegionName                                                
Alabama       Adamsville                   6.903333e+04  6.916667e+04   
              Alabaster                    1.221333e+05  1.230667e+05   
              Albertville                  7.396667e+04  7.260000e+04   
              Arab                         8.376667e+04  8.156667e+04   
              Ardmore                               NaN           NaN   
              Axis                                  NaN           NaN   
              Baileyton                             NaN           NaN   
              Bay Minette                  8.170000e+04  7.853333e+04   
              Bayou La Batre               4.406667e+04  4.450000e+04   
              Bessemer                              NaN           NaN   
              Birmingham                   5.403333e+04  5.440000e+04   
              Boaz                         7.086667e+04  7.026667e+04   
              Brent                        9.293333e+04  9.433333e+04   
              Brighton                              NaN           NaN   
              Brookwood                    9.256667e+04  9.510000e+04   
              Buhl                         9.080000e+04  9.460000e+04   
              Calera                       1.089333e+05  1.103667e+05   
              Center Point                 8.096667e+04  8.123333e+04   
              Centreville                  9.530000e+04  9.656667e+04   
              Chalkville                   9.410000e+04  9.443333e+04   
              Chancellor                            NaN           NaN   
              Chelsea                      1.620667e+05  1.670333e+05   
              Chickasaw                    5.120000e+04  5.366667e+04   
              Chunchula                    8.026667e+04  8.176667e+04   
              Citronelle                   6.483333e+04  6.663333e+04   
              Clay                         1.209000e+05  1.222667e+05   
              Coden                        6.260000e+04  6.480000e+04   
              Coker                        1.181000e+05  1.207667e+05   
              Concord                      7.860000e+04  7.870000e+04   
              Cottondale                   1.008333e+05  1.026333e+05   
              Crane Hill                            NaN           NaN   
              Creola                       7.706667e+04  7.823333e+04   
              Cullman                               NaN           NaN   
              Daphne                       1.231667e+05  1.202333e+05   
              Dauphin Island               1.777000e+05  1.764667e+05   
              Deatsville                   1.127333e+05  1.135000e+05   
              Decatur                               NaN           NaN   
              Duncanville                  1.081000e+05  1.120333e+05   
              Edgewater                             NaN           NaN   
              Eight Mile                   6.900000e+04  6.903333e+04   
              Elberta                      1.347000e+05  1.470333e+05   
              Elmore                                NaN           NaN   
              Fairfield                    5.933333e+04  5.910000e+04   
              Fairhope                     1.614000e+05  1.619333e+05   
              Foley                        1.002333e+05  1.061000e+05   
              Forestdale                   8.896667e+04  8.950000e+04   
              Fosters                      1.075000e+05  1.102000e+05   
              Fultondale                   9.006667e+04  8.836667e+04   
              Gardendale                   1.142000e+05  1.143667e+05   
              Good Hope                             NaN           NaN   
              Grand Bay                    7.343333e+04  7.770000e+04   
              Grant                        8.506667e+04  8.233333e+04   
              Grayson Valley               8.810000e+04  8.936667e+04   
              Graysville          

In [16]:
def run_ttest():
    
    
     
    towns = get_list_of_university_towns()
    startdate = get_recession_start()
    bottomdate = get_recession_bottom()
    houses = convert_housing_data_to_quarters()
    
    houses = houses.reset_index()
    houses['recession_diff'] = houses[startdate] - houses[bottomdate]
    
    towns_houses = pd.merge(houses, towns, how='inner', on=['State', 'RegionName'])
    towns_houses['ctown'] = True
    houses = pd.merge(houses, towns_houses, how='outer', on = ['State', 'RegionName',
                                                              bottomdate, startdate, 
                                                              'recession_diff'])
    houses['ctown'] = houses['ctown'].fillna(False)
    unitowns = houses[houses['ctown'] == True]
    not_unitowns = houses[houses['ctown'] == False]
    
    t, p = ttest_ind(unitowns['recession_diff'].dropna(), not_unitowns['recession_diff'].dropna())
    different = True if p < 0.01 else False
    betters = "university town" if unitowns['recession_diff'].mean() < not_unitowns['recession_diff'].mean() else "non-university town"
    
    return different,p,betters

    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    

run_ttest()




(True, 0.0043252148535112009, 'university town')